In [19]:
import numpy as np
import pandas as pd
#import tensorflow as tf
import os
import json
import re
import fasttext
import time
from jamo import h2j, j2hcj
from konlpy.tag import Mecab
from gensim.test.utils import common_texts
import pickle
#from gensim.models import FastText
from gensim.test.utils import get_tmpfile

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy import sparse

In [29]:
os.listdir()

['.DS_Store',
 'Untitled.ipynb',
 'Petition_bloging.ipynb',
 'Lionlp',
 'model_150.vec',
 'petitions_dataset',
 'petition_model.bin',
 'petitions.ipynb',
 'model.vec',
 'petition_analysis.ipynb',
 'petitions_word.bin',
 'petitions_text.txt',
 '카테고리 분류 로직.key',
 'LionFDC.ipynb',
 '.ipynb_checkpoints',
 'petitions_word_model.bin',
 'Clustering.ipynb']

In [ ]:
FT = fasttext.train_unsupervised('./train_data.txt',minn=3, dim=200)

In [5]:
#FT.save_model('./fast_model_200.bin')

In [20]:
FT = fasttext.load_model('./fast_model_200.bin')

In [12]:
np.mean(np.array([FT.get_word_vector(x) for x in ["문재인", "대통령", "청소년"]]) , axis=0)


array([ 0.48917612,  0.14619029, -0.23441851, -0.15575711,  0.11024643,
       -0.16636968,  0.00168034, -0.16004597,  0.15117146,  0.19096746,
        0.08445033, -0.01390644, -0.16607846,  0.25228772, -0.29893687,
        0.25169957,  0.10236728,  0.07172604,  0.33616063,  0.17920144,
       -0.46488646,  0.06103731, -0.09763602,  0.21636842, -0.42361322,
       -0.20729645,  0.3046976 ,  0.30164203, -0.04207054, -0.11264837,
        0.04694601, -0.13755715, -0.13625656,  0.01722612, -0.39490613,
       -0.17132227,  0.52520007, -0.05777226, -0.19651179,  0.00665094,
       -0.16375932,  0.34068576, -0.38701165, -0.21389651,  0.14074342,
       -0.2546034 ,  0.24223083, -0.03376384, -0.05581504, -0.07596498,
       -0.25401774, -0.20603819,  0.29554808,  0.34350237,  0.01310087,
       -0.26911268, -0.1056411 ,  0.10011196,  0.11456782,  0.04028519,
        0.29919162, -0.01098875,  0.06072019,  0.10988259,  0.11155796,
       -0.5002391 , -0.23043011,  0.14328904, -0.05896452,  0.51

In [7]:
data

NameError: name 'data' is not defined

In [70]:
np.mean(model.wv[['대통령','문재인']],axis=0).shape

<ipython-input-70-9e759fd0b6d3>:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  np.mean(model.wv[['대통령','문재인']],axis=0).shape


(150,)

In [22]:
vecs = []
texts = []
category = []

In [23]:
f = open("data.txt", 'r')
idx = 0
while True:
    line = f.readline().split(' ')
    center_vec = np.mean(np.array([FT.get_word_vector(x) for x in line[1:]]) , axis=0)
    category.append(line[0])
    vecs.append(center_vec)
    texts.append(' '.join(line[1:]))
    idx += 1
    if(idx % 10000 == 0):
        print(idx)
    if len(line) < 2: break
f.close()

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000


/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3334: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [24]:
data = pd.DataFrame()
data['vec'] = vecs
data['text'] = texts
data['category'] = category

In [28]:
np.array(list(data['vec'])).shape

(360805,)

In [35]:
data = data.drop(360804)
data.to_csv('./cap_data.csv')

AttributeError: 'list' object has no attribute 'shape'

In [36]:
for category in list(set(data['category'])):
    km = KMeans(n_clusters=250)
    tmp = data[data['category'] == category]
    km.fit(np.array(list(tmp['vec'])))
    tmp['label'] = km.labels_
    tmp.to_csv('./'+category+'_result.csv')

/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWa

/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWa

/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWa

/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWa

/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWa

...

/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWa

/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
<ipython-input-36-416886e4f5a9>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['label'] = km.labels_
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances

/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWa

/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWa

/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWa

In [37]:
q = pd.read_csv('./교통건축국토_result.csv')

In [40]:
q[q['label'] == 3]

,Unnamed: 0,vec,text,category,label
532,13866,[ 0.00507703 0.21573602 -0.04795677 0.104516...,ktx는 완행 입석제한을 두고 했음 하네요 가뜩이나 비좁은 기차안인데 사람으로 가득...,교통건축국토,3
759,17221,[ 5.61870299e-02 2.14072958e-01 -8.51111263e-...,운전중 적재물 낙하시 누구 한테 물어 달라 해야 되나요 안녕하세요 저희 나라 법이 ...,교통건축국토,3
1064,26652,[ 1.59775943e-03 2.57595152e-01 -1.11291176e-...,수준미달 부실교육 저질 운전학원은 다 없애버려야 합니다 학원비는 드럽게 많이 쳐받아...,교통건축국토,3
1087,27143,[ 0.04479916 0.17397398 -0.09445534 -0.036046...,제발 포항을 내버려두세요 1지열발전소 2CO2 심해 저장소 3포항댐건설 아무것도 하...,교통건축국토,3
1120,28342,[-0.00391866 0.13563064 -0.05598372 -0.038731...,포항지열발전소 포항입니다 발전소 폐쇄 언제하나요 1프로라도 지진가능성이 있다면 답은...,교통건축국토,3
...,...,...,...,...,...
25427,345722,[ 0.01774987 0.21149254 -0.00642941 -0.080280...,디젤차량 통제에 불만 있으신분들 모여서 행정소송합시다 만들어 놓고 쓰지말라니요 이래...,교통건축국토,3
25466,346465,[-2.63117906e-02 1.24157786e-01 -1.62199482e-...,주정차위반 견인료 주정차위반 과태료는 낸다지만 5분거리에 견인해갔으면서 견인료를 2...,교통건축국토,3
25516,347628,[ 3.22484784e-02 1.37513489e-01 -1.60173438e-...,부당한 자동차종합검사비 자동차 종합검사비라 고 인구 50만 이상은 또 5만원이넘게 ...,교통건축국토,3
25682,349979,[-0.01520542 0.14108571 -0.07202817 0.105128...,법인차량운행일지 폐지 법인 차량을 소유한 대표자들 직원분들을 보아하니 실질적으로 정...,교통건축국토,3


In [ ]:
data = pd.read_csv('./cap_data.csv')

In [7]:
np.array(list(data['vec'])).shape

(360804, 200)

In [12]:
km = KMeans(n_clusters=4000)

In [13]:
km.fit(np.array(list(data['vec'])))

KeyboardInterrupt: 

In [ ]:
data['label'] = km.labels

In [ ]:
data.to_csv('./result.csv')

In [61]:
data['vec']

0         [0.040916067, 0.21413994, -0.0108013805, 0.041...
1         [-0.030167144, 0.08921375, -0.012558518, 0.063...
2         [0.005727782, 0.20674542, -0.16998124, -0.0161...
3         [-0.0037659763, 0.13166492, -0.043711826, 0.04...
4         [-0.1189272, 0.31296474, -0.028589422, -0.0634...
                                ...                        
360799    [-0.03701256, 0.22769348, -0.05807231, -0.0502...
360800    [-0.008998756, 0.3166273, -0.10653005, 0.05112...
360801    [0.02047793, 0.26018474, -0.117496416, 0.00815...
360802    [-0.014785209, 0.13325226, 0.008976319, 0.0351...
360803    [0.036441185, 0.15028018, -0.1335398, 0.034774...
Name: vec, Length: 360804, dtype: object